# NGC link

https://docs.nvidia.com/deeplearning/dgx/support-matrix/index.html#framework-matrix-2019

https://developer.nvidia.com/nccl

https://eng.uber.com/horovod/

# CNN Examples

In [ ]:
!cd /workspace && ls -lag

In [ ]:
!cd /workspace/nvidia-examples/ && ls -lag

In [ ]:
!cd /workspace/nvidia-examples/cnn/ && ls -lag

In [1]:
cd /workspace/nvidia-examples/cnn/

/opt/tensorflow/nvidia-examples/cnn


# Alexnet

In [2]:
!cat ./alexnet.py

#!/usr/bin/env python
# Copyright 2018 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

from __future__ import print_function
from builtins import range
import nvutils
import tensorflow as tf

nvutils.init()

default_args = {
    'image_width' : 227,
    'image_height' : 227,
    'image_format' : 'channels_first',
    'distort_color' : False,
    'batch_size' : 256,
    'da

In [3]:
!./alexnet.py 2>/tmp/errors

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --iter_unit epoch
  --num_iter 91
  --batch_size 256
  --precision fp16
  --predict False
  --display_every 10
Training
  Step Epoch Img/sec   Loss  LR
     1   1.0    83.1  6.915  9.988 1.00000
    10  10.0   727.0  5.821  8.886 0.81198
    20  20.0  6620.7  5.560  8.604 0.62601
    30  30.0  6626.0  5.557  8.562 0.46420
    40  40.0  6593.3  5.519  8.479 0.32653
    50  50.0  6613.8  5.506  8.425 0.21302
    60  60.0  6600.9  5.504  8.390 0.12366
    70  70.0  6619.2  5.428  8.293 0.05845
    80  80.0  6630.5  5.475  8.330 0.01739
    90  90.0  6623.0  5.428  8.281 0.00048


In [8]:
!./alexnet.py -h

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
--------------------------------------------------------------------------
[[46993,1],0]: A high-performance Open MPI point-to-point messaging module
was unable to find any relevant network interfaces:

Module: OpenFabrics (openib)
  Host: 566037718bdc

Another transport will be used instead, although this may result in
lower performance.

NOTE: You can disable this warning by setting the MCA parameter
btl_base_warn_component_unused to 0.
--------------------------------------------------------------------------
usage: alexnet.py [-h] [--data_dir DATA_DIR] [--data_idx_dir DATA_IDX_DIR]
                  [-b BATCH_SIZE] [-i NUM_ITER] [-u {epoch,batch}]
                  [--log_dir LOG_DIR] [--export_dir EXPORT_DIR]
                  [--display_every DISPLAY_EVERY] [--precision {fp32,fp16}]
                  [--use_dali [{CPU,GPU}]] [--predict]

optional arguments:
  -h, --help            show this help message and

In [4]:
!./alexnet.py --data_dir /datasets/imagenet_TFrecords 2>/tmp/errors

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --data_dir /datasets/imagenet_TFrecords
  --num_iter 91
  --precision fp16
  --predict False
  --display_every 10
  --iter_unit epoch
  --batch_size 256
Training
  Step Epoch Img/sec   Loss  LR
     1   0.0    69.1  6.912  9.986 1.00000
    10   0.0   767.1  6.902  9.937 0.99996
    20   0.0  4190.6  6.917  9.809 0.99992
    30   0.0  4468.4  6.901  9.573 0.99987
    40   0.0  4486.9  6.909  9.342 0.99983
    50   0.0  4421.6  6.903  9.106 0.99978
    60   0.0  4461.0  6.912  8.902 0.99974
    70   0.0  4049.0  6.907  8.703 0.99970
    80   0.0  4163.8  6.907  8.526 0.99965
    90   0.0  4109.9  6.910  8.370 0.99961
   100   0.0  4287.0  6.909  8.225 0.99957
   110   0.0  4367.6  6.911  8.097 0.99952
   120   0.0  4312.2  6.909  7.978 0.99948
   130   0.0  4303.7  6.911  7.874 0.99943
   140   0.0  4408.3  6.910  7.779 0.99939
   150   0.0  4395.5  6.909  7.692 0.99935
   160   0.0  4366.8  6.

# InceptionV3

In [5]:
!./inception_v3.py --data_dir /datasets/imagenet_TFrecords 2>/tmp/errors

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --data_dir /datasets/imagenet_TFrecords
  --num_iter 90
  --precision fp16
  --iter_unit epoch
  --display_every 10
  --batch_size 128
  --predict False
Training
  Step Epoch Img/sec   Loss  LR
     1   0.0     8.8  7.080  8.192 1.00000
    10   0.0    71.0  7.202  8.315 0.99998
    20   0.0   459.9  7.031  8.146 0.99996
    30   0.0   459.3  7.006  8.119 0.99994
    40   0.0   462.8  6.908  8.014 0.99991
    50   0.0   461.6  6.955  8.049 0.99989
    60   0.0   461.7  6.965  8.043 0.99987
    70   0.0   459.7  6.934  7.995 0.99985
    80   0.0   461.1  6.920  7.962 0.99982
    90   0.0   463.3  6.928  7.952 0.99980
   100   0.0   461.6  6.941  7.946 0.99978
   110   0.0   462.0  6.871  7.859 0.99976
   120   0.0   464.4  6.837  7.808 0.99974
   130   0.0   464.4  6.868  7.822 0.99971
   140   0.0   459.5  6.842  7.779 0.99969
   150   0.0   463.3  6.787  7.708 0.99967
   160   0.0   461.9  6.

# Resnet

In [6]:
!./resnet.py --layers=50 --data_dir=/datasets/imagenet_TFrecords 2>/tmp/errors

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --iter_unit epoch
  --data_dir /datasets/imagenet_TFrecords
  --num_iter 90
  --layers 50
  --precision fp16
  --predict False
  --batch_size 256
  --display_every 10
Training
  Step Epoch Img/sec   Loss  LR
     1   0.0    22.8  7.749  8.720 2.00000
    10   0.0   251.3  7.721  8.692 1.99992
    20   0.0   668.5  7.520  8.495 1.99983
    30   0.0   861.1  7.216  8.195 1.99974
    40   0.0   861.0  7.322  8.302 1.99965
    50   0.0   860.8  7.026  8.002 1.99956
    60   0.0   860.3  6.978  7.943 1.99948
    70   0.0   864.1  6.889  7.838 1.99939
    80   0.0   861.7  6.931  7.859 1.99930
    90   0.0   859.6  6.903  7.810 1.99921
   100   0.0   860.4  6.841  7.725 1.99912
   110   0.0   858.6  6.832  7.693 1.99903
   120   0.0   863.8  6.712  7.551 1.99894
   130   0.0   858.2  6.709  7.526 1.99885
   140   0.0   861.2  6.757  7.552 1.99877
   150   0.0   858.7  6.751  7.524 1.99868
   160   0

In [9]:
!./resnet.py --layers=50 --data_dir=/datasets/imagenet_TFrecords --precision=fp32 2>/tmp/errors

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --batch_size 256
  --predict False
  --display_every 10
  --precision fp32
  --iter_unit epoch
  --layers 50
  --data_dir /datasets/imagenet_TFrecords
  --num_iter 90
Training
  Step Epoch Img/sec   Loss  LR


In [10]:
!./resnet.py --layers=50 --batch_size=128 --data_dir=/datasets/imagenet_TFrecords --precision=fp32 2>/tmp/errors

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --data_dir /datasets/imagenet_TFrecords
  --batch_size 128
  --display_every 10
  --predict False
  --layers 50
  --iter_unit epoch
  --precision fp32
  --num_iter 90
Training
  Step Epoch Img/sec   Loss  LR
     1   0.0    17.7  7.588  8.559 2.00000
    10   0.0   139.2  7.948  8.920 1.99996
    20   0.0   341.3  7.625  8.602 1.99992
    30   0.0   390.9  7.559  8.540 1.99987
    40   0.0   390.7  7.360  8.345 1.99983
    50   0.0   391.8  7.067  8.053 1.99978
    60   0.0   392.9  6.996  7.977 1.99974
    70   0.0   392.2  7.010  7.982 1.99969
    80   0.0   392.8  7.030  7.988 1.99965
    90   0.0   392.8  6.956  7.899 1.99960
   100   0.0   391.5  6.885  7.811 1.99956
   110   0.0   390.4  6.904  7.811 1.99952
   120   0.0   392.1  6.891  7.779 1.99947
   130   0.0   391.3  6.892  7.761 1.99943
   140   0.0   390.7  6.809  7.659 1.99938
   150   0.0   391.5  6.890  7.720 1.99934
   160   0

# DALI with Resnet

In [11]:
!./resnet.py --layers=50 --data_dir=/datasets/imagenet_TFrecords --precision=fp16 --data_idx_dir /datasets/imagenet_idx --use_dali GPU 2>/tmp/error

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --predict False
  --num_iter 90
  --batch_size 256
  --layers 50
  --data_dir /datasets/imagenet_TFrecords
  --use_dali GPU
  --precision fp16
  --display_every 10
  --iter_unit epoch
  --data_idx_dir /datasets/imagenet_idx
Training
  Step Epoch Img/sec   Loss  LR
     1   0.0    22.4  7.502  8.473 2.00000
    10   0.0   278.6  7.497  8.469 1.99992
    20   0.0   820.5  7.483  8.457 1.99983
    30   0.0   836.0  7.198  8.173 1.99974
    40   0.0   829.7  7.095  8.066 1.99965
    50   0.0   837.3  6.939  7.901 1.99956
    60   0.0   834.0  6.980  7.925 1.99948
    70   0.0   837.1  6.939  7.863 1.99939
    80   0.0   835.5  6.964  7.865 1.99930
    90   0.0   830.6  6.837  7.714 1.99921
   100   0.0   840.4  6.811  7.666 1.99912
   110   0.0   840.6  6.849  7.682 1.99903
   120   0.0   833.9  6.819  7.630 1.99894
   130   0.0   838.0  6.718  7.506 1.99885
   140   0.0   838.8  6.715  7.481 1.99

# DALI with Alexnet

In [12]:
!./alexnet.py --batch_size 256 --data_dir=/datasets/imagenet_TFrecords --precision=fp16 --data_idx_dir /datasets/imagenet_idx --use_dali GPU 2>/tmp/error

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --num_iter 91
  --precision fp16
  --data_idx_dir /datasets/imagenet_idx
  --predict False
  --iter_unit epoch
  --data_dir /datasets/imagenet_TFrecords
  --use_dali GPU
  --display_every 10
  --batch_size 256
Training
  Step Epoch Img/sec   Loss  LR
     1   0.0    65.2  6.931 10.006 1.00000
    10   0.0   789.2  6.912  9.946 0.99996
    20   0.0  1533.8  6.909  9.811 0.99992
    30   0.0  3052.9  6.911  9.603 0.99987
    40   0.0  5697.7  6.904  9.362 0.99983
    50   0.0  5760.4  6.906  9.134 0.99978
    60   0.0  5836.6  6.909  8.924 0.99974
    70   0.0  5658.5  6.916  8.735 0.99970
    80   0.0  5617.8  6.911  8.552 0.99965
    90   0.0  5723.0  6.907  8.387 0.99961
   100   0.0  5765.5  6.913  8.247 0.99957
   110   0.0  5739.6  6.910  8.113 0.99952
   120   0.0  5740.4  6.909  7.994 0.99948
   130   0.0  5764.5  6.909  7.887 0.99943
   140   0.0  5752.4  6.906  7.789 0.99939
   150   0

  1830   0.4  5806.6  6.908  6.909 0.99198
  1840   0.4  5875.5  6.906  6.907 0.99194
  1850   0.4  5896.5  6.909  6.910 0.99190
  1860   0.4  5882.3  6.903  6.904 0.99185
  1870   0.4  5880.1  6.908  6.909 0.99181
  1880   0.4  5911.7  6.909  6.910 0.99176
  1890   0.4  5937.0  6.907  6.908 0.99172
  1900   0.4  5901.5  6.909  6.910 0.99168
  1910   0.4  5802.8  6.909  6.910 0.99163
  1920   0.4  5850.5  6.906  6.907 0.99159
  1930   0.4  5879.2  6.904  6.905 0.99155
  1940   0.4  5887.9  6.910  6.911 0.99150
  1950   0.4  5874.8  6.904  6.905 0.99146
  1960   0.4  5862.9  6.909  6.910 0.99141
  1970   0.4  5874.9  6.905  6.906 0.99137
  1980   0.4  5861.4  6.905  6.906 0.99133
  1990   0.4  5889.3  6.908  6.909 0.99128
  2000   0.4  5911.1  6.905  6.906 0.99124
  2010   0.4  5831.5  6.906  6.907 0.99120
  2020   0.4  5827.6  6.908  6.909 0.99115
  2030   0.4  5841.7  6.907  6.908 0.99111
  2040   0.4  5895.4  6.907  6.908 0.99106
  2050   0.4  5905.6  6.906  6.907 0.99102
  2060   0.

  3740   0.7  5875.9  6.906  6.907 0.98365
  3750   0.7  5880.1  6.906  6.907 0.98360
  3760   0.8  5849.8  6.908  6.908 0.98356
  3770   0.8  5836.0  6.906  6.907 0.98351
  3780   0.8  5822.4  6.909  6.909 0.98347
  3790   0.8  5823.1  6.911  6.912 0.98343
  3800   0.8  5859.2  6.910  6.911 0.98338
  3810   0.8  5867.4  6.908  6.909 0.98334
  3820   0.8  5870.9  6.910  6.910 0.98330
  3830   0.8  5905.4  6.912  6.913 0.98325
  3840   0.8  5903.2  6.912  6.912 0.98321
  3850   0.8  5826.0  6.910  6.910 0.98317
  3860   0.8  5835.2  6.910  6.910 0.98312
  3870   0.8  5855.6  6.908  6.909 0.98308
  3880   0.8  5875.6  6.908  6.908 0.98304
  3890   0.8  5737.2  6.908  6.908 0.98299
  3900   0.8  5809.7  6.906  6.906 0.98295
  3910   0.8  5815.8  6.907  6.908 0.98291
  3920   0.8  5780.7  6.909  6.909 0.98286
  3930   0.8  5788.7  6.902  6.903 0.98282
  3940   0.8  5853.0  6.907  6.907 0.98277
  3950   0.8  5835.3  6.910  6.910 0.98273
  3960   0.8  5787.4  6.909  6.909 0.98269
  3970   0.

  5650   1.1  5858.5  6.905  6.906 0.97534
  5660   1.1  5851.9  6.906  6.906 0.97530
  5670   1.1  5809.9  6.910  6.910 0.97526
  5680   1.1  5793.8  6.906  6.906 0.97521
  5690   1.1  5810.4  6.906  6.907 0.97517
  5700   1.1  5833.0  6.905  6.906 0.97513
  5710   1.1  5772.1  6.904  6.904 0.97508
  5720   1.1  5803.1  6.907  6.907 0.97504
  5730   1.1  5826.6  6.911  6.911 0.97500
  5740   1.1  5862.6  6.905  6.906 0.97495
  5750   1.1  5918.9  6.903  6.904 0.97491
  5760   1.2  5864.7  6.909  6.909 0.97487
  5770   1.2  5812.3  6.908  6.909 0.97482
  5780   1.2  5822.5  6.908  6.908 0.97478
  5790   1.2  5817.4  6.905  6.905 0.97474
  5800   1.2  5839.1  6.904  6.905 0.97469
  5810   1.2  5873.7  6.906  6.907 0.97465
  5820   1.2  5894.9  6.907  6.907 0.97461
  5830   1.2  5876.4  6.906  6.906 0.97456
  5840   1.2  5861.3  6.908  6.909 0.97452
  5850   1.2  5913.2  6.903  6.903 0.97448
  5860   1.2  5818.5  6.905  6.905 0.97443
  5870   1.2  5793.1  6.906  6.907 0.97439
  5880   1.

# Multi GPU with Resnet

In [13]:
!mpiexec --allow-run-as-root --bind-to socket -np 8 /workspace/nvidia-examples/cnn/resnet.py --layers=50 --data_dir=/datasets/imagenet_TFrecords --precision=fp16 --data_idx_dir /datasets/imagenet_idx --use_dali GPU 2>/tmp/error

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --data_dir /datasets/imagenet_TFrecords
  --use_dali GPU
  --num_iter 90
  --display_every 10
  --layers 50
  --predict False
  --iter_unit epoch
  --batch_size 256
  --precision fp16
  --data_idx_dir /datasets/imagenet_idx
Training
Training
Training
Training
Training
Training
Training
Training
  Step Epoch Img/sec   Loss  LR
     1   0.0   150.9  7.473  8.444 2.00000
    10   0.0  150